<a href="https://colab.research.google.com/github/SeunghyeChae/tmdhey/blob/main/input_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow_io

     |████████████████████████████████| 23.4 MB 1.6 MB/s 


In [2]:
!pip install pydicom

     |████████████████████████████████| 2.0 MB 7.3 MB/s 


In [3]:
import tensorflow as tf
import pydicom as dcm
import pathlib
import os
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


In [3]:
import os
from tqdm import tqdm
from glob import glob
import random
import tensorflow as tf

dataset_path= '/content/drive/MyDrive/cakd_colab/python_practice/smc/1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260'

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def make_example(img_str, source_id, filename):
    # Create a dictionary with features that may be relevant.
    feature = {'image/source_id': _int64_feature(source_id),
               'image/filename': _bytes_feature(filename),
               'image/encoded': _bytes_feature(img_str),
               # -----------------------------------------
               'image/height': _int64_feature(shape[0]),
               'image/width': _int64_feature(shape[1]),
               'image/channels': _int64_feature(shape[2]),
               'image/shape': _int64_feature(shape),
               'image/image_data':_bytes_feature(image_data.tostring()),
               'image/superpixels':_bytes_feature(superpixels.tostring()),
               'image/mask_instance':_bytes_feature(mask_instance.tostring()),
               'image/mask_class':_bytes_feature(mask_class.tostring()),
               'image/class_labels':_int64_feature(class_labels),
               'image/instance_labels':_int64_feature(instance_labels)}

    return tf.train.Example(features=tf.train.Features(feature=feature))


def main(dataset_path, output_path):
    samples = []
    print("Reading data list...")
    for id_name in tqdm(os.listdir(dataset_path)):
        img_paths = glob(os.path.join(dataset_path, id_name, '*.dcm'))
        for img_path in img_paths:
            filename = os.path.join(id_name, os.path.basename(img_path))
            samples.append((img_path, id_name, filename))
    random.shuffle(samples)

    print("Writing tfrecord file...")
    with tf.io.TFRecordWriter(output_path) as writer:
        for img_path, id_name, filename in tqdm(samples):
            tf_example = make_example(img_str=open(img_path, 'rb').read(),
                                      source_id=int(id_name),
                                      filename=str.encode(filename))
            writer.write(tf_example.SerializeToString())


if __name__ == "__main__":
    main(dataset_path, "kface_bin.tfrecord")

Reading data list...


100%|██████████| 121/121 [00:00<00:00, 6230.57it/s]


Writing tfrecord file...


0it [00:00, ?it/s]


In [13]:
import tensorflow_io as tfio

def _parse_tfrecord():
    def parse_tfrecord(tfrecord):
        features = {'image/source_id': tf.io.FixedLenFeature([], tf.int64),
                    'image/filename': tf.io.FixedLenFeature([], tf.string),
                    'image/encoded': tf.io.FixedLenFeature([], tf.string)}
        x = tf.io.parse_single_example(tfrecord, features)
        x_train = tf.image.decode_jpeg(x['image/encoded'], channels=3)

        y_train = tf.cast(x['image/source_id'], tf.float32)
        x_train = _transform_images()(x_train)
        y_train = _transform_targets(y_train)
        return (x_train, y_train), y_train
    return parse_tfrecord


def _transform_images():
    def transform_images(x_train):
        x_train = tf.image.resize(x_train, (128, 128))
        x_train = tf.image.random_crop(x_train, (112, 112, 3))
        x_train = tf.image.random_flip_left_right(x_train)
        x_train = tf.image.random_saturation(x_train, 0.6, 1.4)
        x_train = tf.image.random_brightness(x_train, 0.4)
        x_train = x_train / 255
        return x_train
    return transform_images


def _transform_targets(y_train):
    return y_train

In [14]:
def load_tfrecord_dataset(tfrecord_name, batch_size, shuffle=True, buffer_size=10240):
    """load dataset from tfrecord"""
    raw_dataset = tf.data.TFRecordDataset(tfrecord_name)
    raw_dataset = raw_dataset.repeat()
    if shuffle:
        raw_dataset = raw_dataset.shuffle(buffer_size=buffer_size)
    dataset = raw_dataset.map(
        _parse_tfrecord(),
        num_parallel_calls=tf.data.experimental.AUTOTUNE
    )
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset

In [16]:
import time 

tfr= '/content/kface_bin.tfrecord'
tf_record= load_tfrecord_dataset(tfr, batch_size=128)
tf_record= iter(tf_record)

start= time.time()
for _ in tqdm(range(121)):
  inputs, labels = next(tf_record)

print(time.time()-start)

  0%|          | 0/121 [00:00<?, ?it/s]


StopIteration: ignored

In [17]:
/content/kface_bin.tfrecord

SyntaxError: ignored

In [ ]:
# imagepath= '/content/drive/MyDrive/cakd_colab/python_practice/smc/1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260'
# if not os.path.exists(imagepath):
#   print(imagepath + 'not exist')
#   return
  
# imgpaths= glob(os.path.join(imagepath,"*.dcm"))
# for imgPath in imgpaths:
#   image_bytes= dcm.dcmread(imgPath)
#   image= image_bytes.pixel_array
#   images.append(image)


In [ ]:
import tensorflow as tf
import pydicom as dcm
import pathlib
import os
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


def encode2TfRecord():
    flag= 1
    imagepath= '/content/drive/MyDrive/cakd_colab/python_practice/smc/1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260'
    if not os.path.exists(imagepath):
        print(imagepath + 'not exist')
        return
    allFulldose= glob(os.path.join(imagepath,"*.dcm"))

    def path2img(imgpaths):
        images= []
        for imgPath in imgpaths:
            image_bytes= dcm.dcmread(imgPath)

            image= image_bytes.pixel_array
            images.append(image)
        return images

    print(len(allFulldose))
    fulldose_imgs = path2img(allFulldose)
    fulldose_imgs = np.array(fulldose_imgs,dtype='int16')
    dataset = tf.data.Dataset.from_tensor_slices((fulldose_imgs))


    def _bytes_feature(value):
        '''Returns a bytes_list from a string / byte.'''
        if  isinstance(value, type(tf.constant(0))):
            value= value.numpy() # ByteList won't unpack a string from an EagerTensor
        return tf.train.Feature(bytes_list= tf.train.BytesList(value=[value]))

    def serialize_exemple(full_img):
        '''Creates a tf.Example message ready to be written to a file'''
        # Creates a dictionary message readyto be written to a file
        # data type

        full_img= full_img.numpy().tobytes()

        feature= {
            'full_img': _bytes_feature(full_img)
        }

        # Create a Features message using tf.train.Example.
        example_proto= tf.train.Example(features= tf.train.Features(feature= feature))
        return example_proto.SerializeToString()
        
    def tf_serialize_example(f):

        tf_string= tf.py_function(
            serialize_exemple,
            f, # pass these args to the above function
            tf.string) # the return type is tf.string
        return tf.reshape(tf_string, ())

    serialized_dataset= dataset.map(tf_serialize_example)

    filename= 'trainData.tfrecord'
    writer= tf.data.experimental.TFRecordWriter(filename)
    writer.write(serialized_dataset)

    print('finished')
    return None

# ----------------------------------------------------------------------------

# def decode(filename):
#     dataset= tf.data.TFRecordDataset(filename)
#     feature= {'full_img': tf.io.FixedLenFeature([], tf.string)}

#     def _parse_example(input):
#         feature_dic= tf.io.parse_single_example(input,feature)
#         feature_dic['full_img']= tf.reshape(tf.io.decode_raw(feature_dic['full_img'], tf.int16),[512,512])
#         return feature_dic['full_img']

#     dataset= dataset.map(_parse_example())

#     def preprocess(full):
#         full= tf.cast(full, dtype=tf.float32)
#         f_min= tf.reduce_min(full)
#         f_max= tf.reduce_max(full)
#         full= (full - f_min) / (f_max - f_min)
#         return full

#     dataset= dataset.map(preprocess)
#     return dataset


encode2TfRecord()

In [18]:
def decode(filename):
    dataset= tf.data.TFRecordDataset(filename)
    feature= {'full_img': tf.io.FixedLenFeature([], tf.string)}

    def _parse_example(input):
        feature_dic= tf.io.parse_single_example(input,feature)
        feature_dic['full_img']= tf.reshape(tf.io.decode_raw(feature_dic['full_img'], tf.int16),[512,512])
        return feature_dic['full_img']

    dataset= dataset.map(_parse_example())

    def preprocess(full):
        full= tf.cast(full, dtype=tf.float32)
        f_min= tf.reduce_min(full)
        f_max= tf.reduce_max(full)
        full= (full - f_min) / (f_max - f_min)
        return full

    dataset= dataset.map(preprocess)
    return dataset


decode('/content/kface_bin.tfrecord')

TypeError: ignored

https://github.com/JZK00/Preprocessing-Tool/blob/0989810c07b73fede39090d2ff72a047ef869f59/MedIA_Processing/convert_dicom_to_png.py

In [ ]:
from __future__ import division
import tensorflow as tf
import numpy as np
import os 
import pydicom
import xml.etree.ElementTree as ET
from PIL import Image, ImageDraw
from skimage.util import img_as_float
from skimage.segmentation import slic
import os
import sys
import random
import scipy.ndimage

# DIRECTORY_IMAGES = './'

# RANDOM_SEED = 4242
# SAMPLES_PER_FILES = 300

# def get_image_data_from_pydicom(dm, q):
#     dm = pydicom.read_file(dm)
#     #x = 1260
#     #y = 910
#     #xscale = x/dm.Rows
#     #yscale = y/dm.Columns
#     image_data = np.array(dm.pixel_array)
#     #image_data = np.float32(image_data)
#     #image_data = scipy.ndimage.interpolation.zoom(image_data, [xscale,yscale])
#     print(image_data.shape)

#     for p in range(175):

#         img_data = image_data[p,:,:,0]
#         #image = img_as_float(image_data)
#         #superpixels = slic(image_data, n_segments = 2000, compactness=0.01, max_iter=10)
#         im = Image.fromarray(img_data)
#         #jpeg_dir = os.makedirs("./datasets/jpeg_%s"%q)
#         ###im.save("./png/png_%s"%q + "out_%s.png"%p)
#         im.save("./png/217a" + "out%s.png" % p)
#     return image_data

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _convert_to_example(image_data):

    """Build an Example proto for an image example.
    Args:
      image_data: string, JPEG encoding of RGB image.
      labels: list of integers, identifier for the ground truth;
      instance: instance labels.
      labels_text: list of strings, human-readable labels.
      mask_instance: numpy matrix of instance.
      mask_class: numpy matrix of class.
      shape: 3 integers, image shapes in pixels.
    Returns:
      Example proto
    """
    example = tf.train.Example(features=tf.train.Features(feature={
        #'image/height': _int64_feature(shape[0]),
        #'image/width': _int64_feature(shape[1]),
        #'image/channels': _int64_feature(shape[2]),
        #'image/shape': _int64_feature(shape),
        'image/image_data':_bytes_feature(image_data.tostring()),
        #'image/superpixels':_bytes_feature(superpixels.tostring()),
        #'image/mask_instance':_bytes_feature(mask_instance.tostring()),
        #'image/mask_class':_bytes_feature(mask_class.tostring()),
        #'image/class_labels':_int64_feature(class_labels),
        #'image/instance_labels':_int64_feature(instance_labels)
    }))
    return example

def _add_to_tfrecord(dataset_dir, name, tfrecord_writer, q):
    """Loads data from image and annotations files and add them to a TFRecord.
    Args:
      dataset_dir: Dataset directory;
      name: Image name to add to the TFRecord;
      tfrecord_writer: The TFRecord writer to use for writing.
    """

    dm = dataset_dir + DIRECTORY_IMAGES + name +'.dcm'
    #xml = dataset_dir + DIRECTORY_ANNOTATIONS + name + '.xml'
    image_data = get_image_data_from_pydicom(dm, q)
    #mask_instance, mask_class, shape, class_labels, class_labels_text, instance_labels = groundtruth_to_mask(xml)
    example = _convert_to_example(image_data)
    tfrecord_writer.write(example.SerializeToString())

def _get_output_filename(output_dir, name, idx):
    return '%s/%s.tfrecord' % (output_dir, name)

def run(dataset_dir, output_dir, name='diameter_measurement', shuffling=False):
    """
    Args:
      dataset_dir: The dataset directory where the dataset is stored.
      output_dir: Output directory.
    """
    if not tf.gfile.Exists(output_dir):
        tf.gfile.MakeDirs(output_dir)
    path = os.path.join(dataset_dir)
    filenames = sorted(os.listdir(path))
    if shuffling:
        random.seed(RANDOM_SEED)
        random.shuffle(filenames)
    i = 0
    fidx = 0
    while i < len(filenames):
    # Open new TFRecord file.
        tf_filename  = _get_output_filename(output_dir, name, fidx)   
        with tf.python_io.TFRecordWriter(tf_filename) as tfrecord_writer:
            j = 0
            #while i < len(filenames) and j < SAMPLES_PER_FILES:
            for q in range(1):
                sys.stdout.write('\r>> Converting image %d/%d' % (i+1, len(filenames)))
                sys.stdout.flush()
                filename = filenames[i]
                img_name = filename[:-4]
                _add_to_tfrecord(dataset_dir, img_name, tfrecord_writer, q)
                i += 1
                j += 1
            fidx += 1
    print('\nFinished converting the diameter measure dataset!')
